In [1]:
import tensorflow as tf
import load_data
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import numpy as np
import pandas as pd
import time

from IPython.display import HTML, clear_output
#import params

In [4]:
# get parameter values
#CMD="python3.6 load_data.py --data_dir $DATADIR --out_label_count 50 --data_serialized \
#    --training --data_mode all --data_batch_size 6 --train_max_steps 1 &> $LOGDIR/$TESTCT.log"

# parameter setup
param_dict= {}
param_dict["data_batch_size"]= 256
param_dict["data_dir"]= './data/train/'
param_dict['data_mode'] = 'all'
param_dict['data_batch_norm']= True
param_dict["data_serialized"]= True
param_dict["data_sparse"]= False
param_dict["adj_sparse"]= False #later on change it to True
param_dict["train_max_steps"]= 1000
param_dict["out_label_count"]= 50
param_dict["is_training"]= True #stop shuffling the data for code check!:) for training you should shuffle the data
# but for the validation set the shuffle off to go through each example once

# gene-gene network parameters
param_dict["network_file_name"]= '9606.STRING_experimental.edge'
param_dict["net_type"]= 'experimental'
param_dict["network_dir"]= './data/network-probe/'

# Graph Convolutional Layer params
#param_dict['GCL_dropout']= 1.
param_dict['num_features_nonzero'] = 0 #(tf.int32) in placeholder
param_dict["l2_weight_decay"]= 0 #5e-4 # 'Weight for L2 loss on embedding matrix.'
param_dict['GCL_batch_norm']= False
param_dict['CL_batch_norm']= False
param_dict['FC_batch_norm']= False

param_dict['Arch'] = [['GC', 15]
                      ['FC', 950],
                      ['FC', 950],
                      ['FC', 950],
                      ['FC', 50]]
        
param_dict["learning_rate"]= 0.00001 #initial learning rate
param_dict["dropout"]= 0.2 # dropout rate for tarin data 

param_dict['probe_length']= 841

param_dict['GraphSage_final_norm']= False
param_dict['residual_mode']= False

In [5]:
#metrics.py module
def masked_softmax_cross_entropy(preds, labels): #, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    #mask = tf.cast(mask, dtype=tf.float32)
    #mask /= tf.reduce_mean(mask)
    #loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels): #, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    
    return tf.reduce_mean(accuracy_all)

# inits.py module
def uniform(shape, scale=0.05, name=None):
    """Uniform init."""
    initial = tf.random_uniform(shape, minval=-scale, maxval=scale, dtype=tf.float32)
    #return tf.get_variable(name, initializer= initial, trainable= True)
    return tf.Variable(initial, name=name)


def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = tf.sqrt(6/(shape[0]+shape[1]))
    init_range= tf.cast(init_range, dtype= tf.float32)
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    #return tf.get_variable(name, initializer= initial, trainable= True)
    return tf.Variable(initial, name=name, validate_shape=False)


def zeros(shape, name=None):
    """All zeros."""
    initial = tf.zeros(shape, dtype=tf.float32)
    #return tf.get_variable(name, initializer= initial, trainable= True)
    return tf.Variable(initial, name=name, validate_shape=False)


def ones(shape, name=None):
    """All ones."""
    initial = tf.ones(shape, dtype=tf.float32)
    #return tf.get_variable(name, initializer= initial, trainable= True)
    return tf.Variable(initial, name=name, validate_shape=False)

#preprocessing functions (later put them in preprocess.py module)
def dense_tensor_to_sparse(dense_tensor):
    a_t = dense_tensor
    # Find indices where the tensor is not zero
    idx = tf.where(tf.not_equal(a_t, 0))
    # Make the sparse tensor
    # Use tf.shape(a_t, out_type=tf.int64) instead of a_t.get_shape()
    # if tensor shape is dynamic
    sparse_tensor = tf.SparseTensor(idx, tf.gather_nd(a_t, idx), tf.shape(a_t, out_type=tf.int64))
    # Make a dense tensor back from the sparse one, only to check result is correct
    #dense = tf.sparse_tensor_to_dense(sparse_tensor)
    return sparse_tensor

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj= adj+tf.eye(tf.shape(adj)[0])
    rowsum= tf.reduce_sum(adj, 1)
    d_inv_sqrt = tf.pow(rowsum, -0.5)
    d_inv_sqrt= tf.reshape(d_inv_sqrt, [1, tf.size(d_inv_sqrt)])
    zero_out= tf.cast(~tf.is_inf(d_inv_sqrt), tf.float32)
    d_inv_sqrt= d_inv_sqrt*zero_out
    d_mat_inv_sqrt = tf.diag(tf.reshape(d_inv_sqrt, [tf.size(d_inv_sqrt)]))
    term1= tf.transpose(tf.matmul(adj, d_mat_inv_sqrt))
    return tf.matmul(term1, d_mat_inv_sqrt)
    
def preprocess_adj(adj,issparse):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj)
    if issparse:
        adj_normalized= dense_tensor_to_sparse(adj_normalized)
    return adj_normalized




###### graph sage adjacency normalization 
def normalize_adj_sage(adj):
    """Symmetrically normalize adjacency matrix."""
    rowsum= tf.reduce_sum(adj, 1)
    d_inv= tf.pow(rowsum, -1)
    d_inv= tf.reshape(d_inv, [1, tf.size(d_inv)])
    zero_out= tf.cast(~tf.is_inf(d_inv), tf.float32)
    d_inv= d_inv*zero_out
    d_mat_inv = tf.diag(tf.reshape(d_inv, [tf.size(d_inv)]))
    return tf.matmul(d_mat_inv, adj)

def preprocess_adj_sage(adj,issparse):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj_sage(adj)
    if issparse:
        adj_normalized= dense_tensor_to_sparse(adj_normalized)
    return adj_normalized
###### end of graph sage adj normalization 

def build_adj(network_dict, issparse):
    graph= nx.from_dict_of_lists(network_dict)
    adj = nx.adjacency_matrix(graph,nodelist= probe_gene_order)
    # convert from scipy sparse to numpy
    adj= adj.toarray()
    adj= tf.convert_to_tensor(adj, dtype= tf.float32)
    adj_new= preprocess_adj_sage(adj,issparse)
    return adj_new, adj


def preprocess_features(features, probe_gene_order, main_gene_order, issparse):
    mask=[]
    for i, ind in enumerate(main_gene_order):
        if ind in probe_gene_order:
            mask.append(1)
        else:
            mask.append(0)
    mask= np.array(mask).reshape(1,len(mask))
    mask= np.concatenate((mask,mask), axis=1)
    
    mask= tf.convert_to_tensor(mask, dtype= tf.float32)
    features= features*mask 
    
    x= features
    intermediate_tensor = tf.reduce_sum(tf.abs(x), 0)
    zero_vector = tf.zeros(shape=(1,1), dtype=tf.float32)
    bool_mask = tf.not_equal(intermediate_tensor, zero_vector)
    #bool_mask should have only one dimension
    bool_mask= tf.reshape(bool_mask, [tf.size(bool_mask)])
    features = tf.boolean_mask(x, bool_mask, axis=1)
    # normalize the features if needed later
    
    # convert the N*(ctrl+pert= 2*G features) tensor to a 3D tensor (N,G,2)
    dim= tf.cast(tf.shape(features)[1]/2, tf.int32)
    features= tf.reshape(features, [tf.shape(features)[0],2,dim])
    features= tf.transpose(features, perm=[0, 2, 1]) # a 3-D tensor. each 2D tensor is a sample 
                         
    # convert dense tensor to sparse tensor
    if issparse:
        features= dense_tensor_to_sparse(features)
    
    # needed for tf.assign()
    features= tf.reshape(features, [param_dict["data_batch_size"], param_dict["probe_length"], 2])
    return features


#neural net classes
# layer.py module
# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}

def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]

def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)

def dot_2D(x, y, sparse=False):
    if sparse:
        res= tf.sparse_tensor_dense_matmul(x,y)
    else:
        res= tf.matmul(x,y)
    return res # res is a 2D tensor

def dot(x, y, sparse=False, is_adj= False):
    """Wrapper for tf.matmul (sparse vs dense).""" 
    if sparse:
        if is_adj:
            # x is the adjacency(sparse) and y is the data (dense)
            orig_size= tf.shape(y)
            y= tf.transpose(y, [1,0,2])
            y= tf.reshape(y, [tf.shape(y)[0], -1])
            res= tf.sparse_tensor_dense_matmul(x,y) # res is a dense matrix
            res= tf.reshape(res, [tf.shape(res)[0],-1,orig_size[2]])
            res= tf.transpose(res,[1,0,2])
        else:
            # y is the w and x is the data which is sparse
            orig_size= tf.shape(x)
            x= tf.sparse_reshape(x, [-1,tf.shape(x)[2]])
            res= tf.sparse_tensor_dense_matmul(x,y)
            res= tf.reshape(res, [-1,orig_size[1],tf.shape(res)[1]])
    else:
        if is_adj:
            # y is the data and x is the adjacency
            #x= tf.tile(tf.expand_dims(x,0),[tf.shape(y)[0],1,1])
            #res = tf.matmul(x, y)
            orig_size= tf.shape(y)
            y= tf.transpose(y, [1,0,2])
            y= tf.reshape(y, [tf.shape(y)[0], -1])
            res= tf.matmul(x,y)
            res= tf.reshape(res, [tf.shape(res)[0],-1,orig_size[2]])
            res= tf.transpose(res,[1,0,2])
        else:
            # y is the w and x is the data
            #y= tf.tile(tf.expand_dims(y,0),[tf.shape(x)[0],1,1])
            #res = tf.matmul(x, y)
            orig_size= tf.shape(x)
            x= tf.reshape(x, [-1,tf.shape(x)[2]])
            res= tf.matmul(x,y)
            res= tf.reshape(res, [-1,orig_size[1],tf.shape(res)[1]])
    return res # res is 3D and dense

class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).
    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off
    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs): #implements function called operator
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])


class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, params, placeholders_val, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        # modify this part later
        if dropout:
            self.dropout = placeholders_val['dropout']
        else:
            self.dropout = 0. # there is no need to initialize it as a tensor scalar

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless #check later!
        self.bias = bias
        self.params= params

        # helper variable for sparse dropout
        self.num_features_nonzero = params['num_features_nonzero'] # num_features_nonzero is a tensor of unknown shape
        # fed through the placeholder
        
        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout) #By default, each element is kept or dropped independently(without noise shape)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)], # you have to implemet dot for 3D x
                              sparse=self.sparse_inputs, is_adj= False)
            else:
                pre_sup = self.vars['weights_' + str(i)]
            support = dot(self.support[i], pre_sup, sparse=False, is_adj= True)
            supports.append(support)
        #output = tf.add_n(supports)
        output= supports[0]
        
        # bias
        if self.bias:
            output += self.vars['bias']  # assuming the output is of size batchNo*row*col and bias is of size (col)
            # the bias will be summed up with each row in the output

        output= self.act(output)
        

        if self.params['GCL_batch_norm']:
            # you need to reshape the output if you want to use this!
            with tf.variable_scope(self.name + '_vars'):
                output= tf.layers.batch_normalization(inputs= output, axis= 0,training= placeholders_val['is_training'], 
                                                      trainable= True, name= 'bn', reuse=tf.AUTO_REUSE)
        
        return output 
    
class GraphSage(Layer):    #not implemented 
    """Graph convolution layer using GraphSage algorithm."""
    def __init__(self, input_dim, output_dim, placeholders, params, placeholders_val, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphSage, self).__init__(**kwargs)

        # modify this part later
        if dropout:
            self.dropout = placeholders_val['dropout']
        else:
            self.dropout = 0. # there is no need to initialize it as a tensor scalar

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless #check later!
        self.bias = bias
        self.params= params
        self.output_dim= output_dim
    
        # helper variable for sparse dropout
        self.num_features_nonzero = params['num_features_nonzero'] # num_features_nonzero is a tensor of unknown shape
        # fed through the placeholder
        
        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([2*input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout) #By default, each element is kept or dropped independently(without noise shape)

        # convolve
        supports = list()
        neighbour_mean = dot(self.support[0], x, sparse=False, is_adj= True)
        # concatenate neighbour mean with node feature itself
        x= tf.concat([neighbour_mean, x], axis= 2)
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)], # you have to implemet dot for 3D x
                              sparse=self.sparse_inputs, is_adj= False)
            else:
                pre_sup = self.vars['weights_' + str(i)]
        output= pre_sup
        
        # bias
        if self.bias:
            output += self.vars['bias']  # assuming the output is of size batchNo*row*col and bias is of size (col)
            # the bias will be summed up with each row in the output

        output= self.act(output)

        if self.params['GCL_batch_norm']:
            output= tf.reshape(output, [self.params['data_batch_size'], 
                                        self.params['probe_length'],
                                        self.output_dim])
            with tf.variable_scope(self.name + '_vars'):
                output= tf.layers.batch_normalization(inputs= output, axis= 0,training= placeholders_val['is_training'], 
                                                      trainable= True, name= 'bn', reuse=tf.AUTO_REUSE)
        
        return output 
    
class Conv_1D(Layer):
    """1D convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, params, placeholders_val, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(Conv_1D, self).__init__(**kwargs)

        # modify this part later
        if dropout:
            self.dropout = placeholders_val['dropout']
        else:
            self.dropout = 0. # there is no need to initialize it as a tensor scalar

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless #check later!
        self.bias = bias
        self.params= params
        self.output_dim= output_dim

        # helper variable for sparse dropout
        self.num_features_nonzero = params['num_features_nonzero'] # num_features_nonzero is a tensor of unknown shape
        # fed through the placeholder
        
        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout) #By default, each element is kept or dropped independently(without noise shape)

        # convolve
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)], # you have to implemet dot for 3D x
                              sparse=self.sparse_inputs, is_adj= False)
            else:
                pre_sup = self.vars['weights_' + str(i)]
        output= pre_sup
        
        # bias
        if self.bias:
            output += self.vars['bias']  # assuming the output is of size batchNo*row*col and bias is of size (col)
            # the bias will be summed up with each row in the output

        output= self.act(output)
        
        if self.params['CL_batch_norm']:
            output= tf.reshape(output, [self.params['data_batch_size'], 
                            self.params['probe_length'],
                            self.output_dim])
            with tf.variable_scope(self.name + '_vars'):
                output= tf.layers.batch_normalization(inputs= output, axis= 0,training= placeholders_val['is_training'], 
                                                      trainable= True, name= 'bn', reuse=tf.AUTO_REUSE)
        return output 

    
    
class Dense(Layer):
    """Dense layer."""
    def __init__(self, input_dim, output_dim, placeholders, params,  placeholders_val, dropout=0., sparse_inputs=False,
                 act=tf.nn.relu, bias=False, featureless=False, is_2D= False, **kwargs):
        super(Dense, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders_val['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias
        self.params= params
        self.is_2D= is_2D
        self.output_dim= output_dim
        
        # helper variable for sparse dropout
        self.num_features_nonzero = params['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim],
                                          name='weights')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs
        
        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # transform
        if not self.is_2D:
            x= tf.reshape(x, [-1,tf.shape(x)[1]*tf.shape(x)[2]]) # later you need to add a flag to check for the input shape
        output = dot_2D(x, self.vars['weights'], sparse=self.sparse_inputs)
        
        # bias
        if self.bias:
            output += self.vars['bias']
        
        output= self.act(output)
        
        if self.params['FC_batch_norm']:
            output= tf.reshape(output, [self.params['data_batch_size'],
                                        self.output_dim])
            with tf.variable_scope(self.name + '_vars'):
                output= tf.layers.batch_normalization(inputs= output, axis= 0,training= placeholders_val['is_training'], 
                                                      trainable= True, name= 'bn', reuse=tf.AUTO_REUSE)
        
        return output
    
# model.py module
class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}
        self.params= {}

        self.layers = []
        self.activations = []
        self.activations_eval = []

        self.inputs = None
        self.inputs_eval = None
        self.outputs = None
        self.outputs_eval = None
        
        self.inter_out= None #to check whether the dropout works for training and validation 
        self.inter_out_eval= None

        self.loss = 0
        self.loss_eval = 0
        
        self.accuracy = 0
        self.accuracy_eval = 0
        
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.inputs = self.placeholders['features']
        self.inputs_norm= self.inputs 
        
        if self.params['data_batch_norm']:
            self.inputs_norm= batch_normalize(self.inputs)
            
        self.activations.append(self.inputs_norm)
        for i, layer in enumerate(self.layers):
            if self.params['residual_mode']:
                if i==2:
                    residual= tf.concat([self.activations[-2], self.activations[-1]], axis=2)
                    hidden = layer(residual)
                else:
                    hidden = layer(self.activations[-1])
            elif self.params['GraphSage_final_norm']:
                if i==2:
                    norm = tf.nn.l2_normalize(self.activations[-1], 2)
                    hidden= layer(norm)
                else:
                    hidden = layer(self.activations[-1])
            else:
                hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)
    
    def parameter_count(self):
        return len(self.vars.keys())
        
    def build_eval(self):
        # Build sequential layer model
        self.inputs_eval = self.placeholders['features_eval']
        self.inputs_norm_eval= self.inputs_eval 
        
        if self.params['data_batch_norm']:
            self.inputs_norm_eval= batch_normalize(self.inputs_eval)
           
        self.activations_eval.append(self.inputs_norm_eval)
        for i, layer in enumerate(self.layers):
            if self.params['residual_mode']:
                if i==2:
                    residual= tf.concat([self.activations_eval[-2], self.activations_eval[-1]], axis=2)
                    hidden = layer(residual)
                else:
                    hidden = layer(self.activations_eval[-1])
            elif self.params['GraphSage_final_norm']:
                if i==2:
                    norm = tf.nn.l2_normalize(self.activations_eval[-1], 2)
                    hidden= layer(norm)
                else:
                    hidden = layer(self.activations_eval[-1])
            else:
                hidden = layer(self.activations_eval[-1])
            self.activations_eval.append(hidden)
        self.outputs_eval = self.activations_eval[-1]

        # Build metrics
        self._loss_eval()
        self._accuracy_eval()


    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError
        
    
    def _loss_eval(self):
        raise NotImplementedError

    def _accuracy_eval(self):
        raise NotImplementedError
        
    def update_field(features_alter, labels_alter):
        pass

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "./model-save/tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None, save_path= '/model-save/tmp/'):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        #save_path = "./visual-save-load/tmp/%s.ckpt" % self.name
        saver.restore(sess, tf.train.latest_checkpoint(save_path))
        print("Model restored from file: %s" % save_path)
           

class GCN(Model):
    def __init__(self, placeholders, params, input_dim, placeholders_val, **kwargs):
        super(GCN, self).__init__(**kwargs)
        
        self.input_dim = input_dim
        #self.input_dim = self.inputs.get_shape().as_list()[2]  # To be supported in future Tensorflow versions
        #self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.output_dim= tf.shape(placeholders['labels'])[1]
        self.placeholders = placeholders
        self.params= params
        self.output_average= 0.
        self.placeholders_val= placeholders_val

        self.optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        self.build()
        self.build_eval()
        

    #def update_field(self, dic={}):
    #    self.placeholders= dic
    #    self.inputs= self.placeholders['features']
        
    def _loss(self):  # should be implemented
        # Weight decay loss
        for var in self.layers[0].vars.values():
            pass
            #self.loss += self.params["l2_weight_decay"] * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'])#,
                                                  #self.placeholders['labels_mask'])
    
    def _loss_eval(self):  # should be implemented
        # Weight decay loss
        for var in self.layers[0].vars.values():
            pass
            #self.loss += self.params["l2_weight_decay"] * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss_eval += masked_softmax_cross_entropy(self.outputs_eval, self.placeholders['labels_eval'])#,
                                                  #self.placeholders['labels_mask'])
    
    def output_avg(self):
        self.out_avg, self.out_std = tf.nn.moments(tf.reshape(self.outputs,[-1]), axes=[0])

    def _accuracy(self): # should be implemented
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels']) #,
                                        #self.placeholders['labels_mask'])
    def _accuracy_eval(self): # should be implemented
        self.accuracy_eval = masked_accuracy(self.outputs_eval, self.placeholders['labels_eval']) #,
                                        #self.placeholders['labels_mask'])

    def _build(self):
        in_dim = self.input_dim
        layer_constructors = {'GC': GraphSage,
                              'CONV1D': Conv_1D,
                              'FC': Dense,
                              'FC_SPEC': Dense}
        
        for i, [layer_type, out_dim] in enumerate(self.params['Arch']):
            assert layer_type.upper() in list(layer_constructors.keys())
            layer_const = layer_constructors[layer_type.upper()]
            
            # cancatenation of the CL and GL features
            if self.params['residual_mode']:
                if i==2:
                    in_dim= 2*in_dim
                    
            kwargs = dict(input_dim= in_dim,
                          output_dim=out_dim,
                          placeholders=self.placeholders,
                          placeholders_val= self.placeholders_val,
                          params= self.params,
                          act=tf.nn.relu, # not sure about activation
                          dropout=True, # not sure about dropout implementation
                          sparse_inputs=False,
                          bias= True,
                          logging=self.logging)
            
            
            #Special accomodations for FC_spec
#             if layer_type.upper() == 'FC_SPEC':
#                 kwargs['input_dim'] = kwargs['input_dim'] * self.params["probe_length"]
#                 kwargs['is_2D'] = False
#             if layer_type.upper() == 'FC':
#                 #if not(self.params['Arch'][i-1][0] == 'FC'):
#                 #    kwargs['input_dim'] = kwargs['input_dim'] * self.params["probe_length"]
#                 #    kwargs['is_2D'] = False
#                 #else:
#                 kwargs['is_2D'] = True # FOR ELSE 
                
#                 if i == (len(self.params['Arch'])-1):
#                     kwargs['act'] = lambda x: x
             
            
             #Special accomodations for FC
            if layer_type.upper() == 'FC':
                if not(self.params['Arch'][i-1][0] == 'FC'):
                    kwargs['input_dim'] = kwargs['input_dim'] * self.params["probe_length"]
                    kwargs['is_2D'] = False
                else:
                    kwargs['is_2D'] = True
                
                if i == (len(self.params['Arch'])-1):
                    kwargs['act'] = lambda x: x
                        
            self.layers.append(layer_const(**kwargs))
            in_dim = out_dim
            

    def predict(self):
        return tf.nn.softmax(self.outputs)


def batch_normalize(orig_values):
    """normalize layer across batch"""
    [mean, var] = tf.nn.moments(orig_values, 0)
    norm_values = tf.nn.batch_normalization(orig_values, mean, var,
                                            offset=None,
                                            scale=None,
                                            variance_epsilon=1e-3)
    return norm_values

# placeholder for validation
def construct_feed_dict(dropout, is_training, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['dropout']: dropout})
    feed_dict.update({placeholders['is_training']: is_training})
    return feed_dict

In [6]:
def count_params(var_list):
    param_counts = []
    for variable in var_list:
        variable_parameters = 1
        shape = variable.get_shape()
        for dim in shape:
            variable_parameters *= dim.value
        param_counts.append(variable_parameters)
    return np.sum(param_counts)

In [7]:
with tf.device('/device:GPU:0'):
    # main body 
    # loading the data (network and node features)
    feat_b, label_b, meta_b, input_size, nummeta, numexamps = load_data.get_batch(param_dict, True)

    print("data final input_size: " + str(input_size), file= open('printout','a'))
    print("data nummeta: " + str(nummeta), file= open('printout','a'))

    # loading the interaction network and probe gene order then bulding adjacency
    saved_list_names= [param_dict["network_file_name"], 
                       'probe_gene_order_'+param_dict["net_type"], 'main_gene_order']
    objects= []
    for i, name in enumerate(saved_list_names):
        infile = open(param_dict["network_dir"]+name+'.pkl',"rb")
        objects.append(pkl.load(infile))
    infile.close()

    network_dict, probe_gene_order, main_gene_order= tuple(objects)

### start load_data
data class_column: 1957
data num_metadata: 24
data num_examples: 50000
data number of datafiles: 1
data example datafile: ./data/train/file0.tfrecord
data batch_size: 256
data nreaders: 1
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
data orig input_size: 1956
data final input_size: 1956


In [8]:
with tf.device('/device:GPU:0'):
    param_dict['train_max_steps']= 100000 #5
    param_dict["save_freq"]= int(50000/param_dict['data_batch_size']) # 1 # 391 # int(50000/param_dict['data_batch_size'])

    # setup for validation dataset 
    identifier= 'eval'
    param_dict_eval= param_dict.copy()
    param_dict_eval["data_dir"]= './data/'+identifier+'/'
    param_dict_eval['eval_max_steps']= int(5000/param_dict_eval['data_batch_size']) #1
    param_dict_eval['dropout']= 0.
    param_dict_eval['is_training']= False
    #param_dict_eval['data_batch_size']= 500

    # building adj matrix (sparse)
    adj_new, adj_prev= build_adj(network_dict, param_dict['adj_sparse'])

    # preprocess the first batch of features
    features_new= preprocess_features(feat_b, probe_gene_order, main_gene_order, param_dict['data_sparse'])
    true_labels_one_hot = tf.one_hot(label_b, depth=param_dict['out_label_count'],
                                             on_value=1., off_value=0., dtype= tf.float32) #  a tensor with shape (sample_size, number of classes)

    # loading the validation data
    feat_eval_b, label_eval_b, meta_eval_b, input_eval_size, nummeta_eval, numexamps_eval = load_data.get_batch(param_dict_eval, False)
    # preprocess the first batch of features
    features_new_eval= preprocess_features(feat_eval_b, probe_gene_order, main_gene_order, param_dict_eval['data_sparse'])
    true_labels_one_hot_eval = tf.one_hot(label_eval_b, depth=param_dict_eval['out_label_count'],
                                             on_value=1., off_value=0., dtype= tf.float32) #  a tensor with shape (sample_size, number of classes)

    # holder dict will be extended later
    holder= {'features': features_new, 'labels': true_labels_one_hot, 'support': [adj_new], 
             'features_eval': features_new_eval, 'labels_eval': true_labels_one_hot_eval}

### start load_data
data class_column: 1957
data num_metadata: 24
data num_examples: 15000
data number of datafiles: 1
data example datafile: ./data/test/file0.tfrecord
data batch_size: 256
data nreaders: 1
data orig input_size: 1956
data final input_size: 1956


In [9]:
############## Validation and test ###############################
with tf.device('/device:GPU:0'):
    placeholders_val= {'dropout': tf.placeholder_with_default(0., shape=()),
                       'is_training': tf.placeholder_with_default(True, shape=())}
    feed_dict= construct_feed_dict(param_dict['dropout'], param_dict['is_training'], placeholders_val)

    # Create model
    model = GCN(holder, param_dict, input_dim=tf.shape(features_new)[2], placeholders_val= placeholders_val, logging=True)
    model.output_avg()
    save_freq= param_dict["save_freq"]
    valid_freq= 391

    

sess= tf.Session(config=tf.ConfigProto(allow_soft_placement= True))    
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)
print("data feat_batch, label_batch, meta_batch: ", file= open('printout','a'))

# initialize the queue threads to start to shovel data
# initialize the variable

ALL_EVAL_ACC= []
ALL_EVAL_LOSS= []


sess.run(tf.initializers.global_variables())
model.load(sess, save_path='./model-save/tmp/')

if identifier == 'test_unseen':
    dev_steps= 5 #int(np.ceil(320/256)) # LCM(256, 320)= 1280
elif identifier == 'test':
    dev_steps= 1875 #int(np.ceil(15000/256)) # LCM(256, 15000)= 480,000
elif identifier == 'eval':
    dev_steps= 625 #int(np.ceil(5000/256)) # LCM(256,5000)= 160,000
    
for step in range(dev_steps):
    # Train model
    t = time.time()
    

    feed_dict.update({placeholders_val['dropout']: param_dict_eval['dropout']})
    feed_dict.update({placeholders_val['is_training']: param_dict_eval['is_training']})
    
    outs_val = sess.run([model.loss_eval, model.accuracy_eval, 
                         model.inputs_eval, model.outputs_eval], feed_dict= feed_dict)
    ALL_EVAL_LOSS.append(outs_val[0])
    ALL_EVAL_ACC.append(outs_val[1])
    
    # Print results
    print("Epoch:", '%04d' % (step + 1),"val_loss= ", "{:.5f}".format(outs_val[0]),
          "val_accuracy= ", "{:.5f}".format(outs_val[1]))
    
print("Optimization Finished!")
print('total validation loss: ', sum(ALL_EVAL_LOSS)/len(ALL_EVAL_LOSS))
print('total validation accuracy: ', sum(ALL_EVAL_ACC)/len(ALL_EVAL_ACC))


# We request our child threads to stop ...
coord.request_stop()
# ... and we wait for them to do so before releasing the main thread
coord.join(threads)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from ./model-save/tmp/gcn.ckpt
Model restored from file: ./model-save/tmp/
Epoch: 0001 val_loss=  1.61343 val_accuracy=  0.52734
Epoch: 0002 val_loss=  1.65511 val_accuracy=  0.51562
Epoch: 0003 val_loss=  1.49273 val_accuracy=  0.57031
Epoch: 0004 val_loss=  1.52894 val_accuracy=  0.56641
Epoch: 0005 val_loss=  1.75918 val_accuracy=  0.47266
Epoch: 0006 val_loss=  1.63992 val_accuracy=  0.55859
Epoch: 0007 val_loss=  1.66304 val_accuracy=  0.54297
Epoch: 0008 val_loss=  1.65712 val_accuracy=  0.50000
Epoch: 0009 val_loss=  1.65382 val_accuracy=  0.52344
Epoch: 0010 val_loss=  1.47999 val_accuracy=  0.53516
Epoch: 0011 val_loss=  1.47463 val_accuracy=  0.57422
Epoch: 0012 val_loss

Epoch: 0139 val_loss=  1.64918 val_accuracy=  0.53906
Epoch: 0140 val_loss=  1.66180 val_accuracy=  0.53125
Epoch: 0141 val_loss=  1.71026 val_accuracy=  0.53906
Epoch: 0142 val_loss=  1.51161 val_accuracy=  0.56641
Epoch: 0143 val_loss=  1.52943 val_accuracy=  0.55859
Epoch: 0144 val_loss=  1.54667 val_accuracy=  0.59375
Epoch: 0145 val_loss=  1.63837 val_accuracy=  0.53125
Epoch: 0146 val_loss=  1.47616 val_accuracy=  0.58594
Epoch: 0147 val_loss=  1.71130 val_accuracy=  0.49219
Epoch: 0148 val_loss=  1.61204 val_accuracy=  0.51562
Epoch: 0149 val_loss=  1.78699 val_accuracy=  0.51953
Epoch: 0150 val_loss=  1.60601 val_accuracy=  0.55078
Epoch: 0151 val_loss=  1.73551 val_accuracy=  0.52344
Epoch: 0152 val_loss=  1.73396 val_accuracy=  0.52344
Epoch: 0153 val_loss=  1.53685 val_accuracy=  0.54688
Epoch: 0154 val_loss=  1.38079 val_accuracy=  0.60156
Epoch: 0155 val_loss=  1.54027 val_accuracy=  0.57031
Epoch: 0156 val_loss=  1.60539 val_accuracy=  0.56250
Epoch: 0157 val_loss=  1.494

Epoch: 0292 val_loss=  1.57541 val_accuracy=  0.57031
Epoch: 0293 val_loss=  1.62727 val_accuracy=  0.53516
Epoch: 0294 val_loss=  1.63073 val_accuracy=  0.52734
Epoch: 0295 val_loss=  1.63411 val_accuracy=  0.51172
Epoch: 0296 val_loss=  1.48906 val_accuracy=  0.57812
Epoch: 0297 val_loss=  1.57348 val_accuracy=  0.55469
Epoch: 0298 val_loss=  1.74700 val_accuracy=  0.48828
Epoch: 0299 val_loss=  1.63292 val_accuracy=  0.55469
Epoch: 0300 val_loss=  1.64273 val_accuracy=  0.55078
Epoch: 0301 val_loss=  1.63121 val_accuracy=  0.50391
Epoch: 0302 val_loss=  1.70680 val_accuracy=  0.50781
Epoch: 0303 val_loss=  1.46070 val_accuracy=  0.54297
Epoch: 0304 val_loss=  1.47387 val_accuracy=  0.57031
Epoch: 0305 val_loss=  1.40854 val_accuracy=  0.55078
Epoch: 0306 val_loss=  1.56250 val_accuracy=  0.53125
Epoch: 0307 val_loss=  1.67122 val_accuracy=  0.51953
Epoch: 0308 val_loss=  1.75111 val_accuracy=  0.51953
Epoch: 0309 val_loss=  1.85437 val_accuracy=  0.48438
Epoch: 0310 val_loss=  1.428

Epoch: 0446 val_loss=  1.54094 val_accuracy=  0.53906
Epoch: 0447 val_loss=  1.34433 val_accuracy=  0.60547
Epoch: 0448 val_loss=  1.52443 val_accuracy=  0.58594
Epoch: 0449 val_loss=  1.60681 val_accuracy=  0.55859
Epoch: 0450 val_loss=  1.51316 val_accuracy=  0.57031
Epoch: 0451 val_loss=  1.68520 val_accuracy=  0.53125
Epoch: 0452 val_loss=  1.36841 val_accuracy=  0.59375
Epoch: 0453 val_loss=  1.63169 val_accuracy=  0.48828
Epoch: 0454 val_loss=  1.44054 val_accuracy=  0.58984
Epoch: 0455 val_loss=  1.63952 val_accuracy=  0.52344
Epoch: 0456 val_loss=  1.60617 val_accuracy=  0.53516
Epoch: 0457 val_loss=  1.53599 val_accuracy=  0.58203
Epoch: 0458 val_loss=  1.61563 val_accuracy=  0.51562
Epoch: 0459 val_loss=  1.78741 val_accuracy=  0.50781
Epoch: 0460 val_loss=  1.53517 val_accuracy=  0.58594
Epoch: 0461 val_loss=  1.68619 val_accuracy=  0.49219
Epoch: 0462 val_loss=  1.53512 val_accuracy=  0.52734
Epoch: 0463 val_loss=  1.42768 val_accuracy=  0.55859
Epoch: 0464 val_loss=  1.633

Epoch: 0598 val_loss=  1.44431 val_accuracy=  0.55078
Epoch: 0599 val_loss=  1.57448 val_accuracy=  0.51953
Epoch: 0600 val_loss=  1.66656 val_accuracy=  0.51172
Epoch: 0601 val_loss=  1.73422 val_accuracy=  0.52734
Epoch: 0602 val_loss=  1.83091 val_accuracy=  0.49219
Epoch: 0603 val_loss=  1.41065 val_accuracy=  0.61328
Epoch: 0604 val_loss=  1.67533 val_accuracy=  0.52734
Epoch: 0605 val_loss=  1.58741 val_accuracy=  0.53906
Epoch: 0606 val_loss=  1.38895 val_accuracy=  0.60938
Epoch: 0607 val_loss=  1.54547 val_accuracy=  0.53516
Epoch: 0608 val_loss=  1.67908 val_accuracy=  0.53125
Epoch: 0609 val_loss=  1.75870 val_accuracy=  0.53125
Epoch: 0610 val_loss=  1.62318 val_accuracy=  0.54688
Epoch: 0611 val_loss=  1.62169 val_accuracy=  0.54688
Epoch: 0612 val_loss=  1.45957 val_accuracy=  0.57812
Epoch: 0613 val_loss=  1.49705 val_accuracy=  0.59766
Epoch: 0614 val_loss=  1.67650 val_accuracy=  0.51953
Epoch: 0615 val_loss=  1.49710 val_accuracy=  0.53516
Epoch: 0616 val_loss=  1.639

...

Epoch: 1207 val_loss=  1.63409 val_accuracy=  0.53906
Epoch: 1208 val_loss=  1.54931 val_accuracy=  0.55859
Epoch: 1209 val_loss=  1.36886 val_accuracy=  0.56641
Epoch: 1210 val_loss=  1.51107 val_accuracy=  0.60938
Epoch: 1211 val_loss=  1.56468 val_accuracy=  0.54688
Epoch: 1212 val_loss=  1.51596 val_accuracy=  0.56250
Epoch: 1213 val_loss=  1.67213 val_accuracy=  0.53125
Epoch: 1214 val_loss=  1.48035 val_accuracy=  0.56641
Epoch: 1215 val_loss=  1.60395 val_accuracy=  0.50391
Epoch: 1216 val_loss=  1.43029 val_accuracy=  0.59375
Epoch: 1217 val_loss=  1.67842 val_accuracy=  0.52344
Epoch: 1218 val_loss=  1.56456 val_accuracy=  0.51172
Epoch: 1219 val_loss=  1.56623 val_accuracy=  0.56641
Epoch: 1220 val_loss=  1.62125 val_accuracy=  0.51172
Epoch: 1221 val_loss=  1.74542 val_accuracy=  0.52344
Epoch: 1222 val_loss=  1.67932 val_accuracy=  0.51172
Epoch: 1223 val_loss=  1.61420 val_accuracy=  0.50000
Epoch: 1224 val_loss=  1.43741 val_accuracy=  0.55469
Epoch: 1225 val_loss=  1.510

Epoch: 1360 val_loss=  1.42438 val_accuracy=  0.58594
Epoch: 1361 val_loss=  1.79986 val_accuracy=  0.44922
Epoch: 1362 val_loss=  1.57761 val_accuracy=  0.55078
Epoch: 1363 val_loss=  1.79434 val_accuracy=  0.50781
Epoch: 1364 val_loss=  1.65551 val_accuracy=  0.54297
Epoch: 1365 val_loss=  1.39645 val_accuracy=  0.60938
Epoch: 1366 val_loss=  1.72484 val_accuracy=  0.52344
Epoch: 1367 val_loss=  1.54833 val_accuracy=  0.57422
Epoch: 1368 val_loss=  1.51669 val_accuracy=  0.56641
Epoch: 1369 val_loss=  1.53839 val_accuracy=  0.57031
Epoch: 1370 val_loss=  1.63503 val_accuracy=  0.52344
Epoch: 1371 val_loss=  1.65882 val_accuracy=  0.58594
Epoch: 1372 val_loss=  1.67508 val_accuracy=  0.53906
Epoch: 1373 val_loss=  1.59873 val_accuracy=  0.56641
Epoch: 1374 val_loss=  1.54638 val_accuracy=  0.58594
Epoch: 1375 val_loss=  1.39959 val_accuracy=  0.61328
Epoch: 1376 val_loss=  1.69207 val_accuracy=  0.51172
Epoch: 1377 val_loss=  1.51502 val_accuracy=  0.55469
Epoch: 1378 val_loss=  1.718

Epoch: 1512 val_loss=  1.59469 val_accuracy=  0.55469
Epoch: 1513 val_loss=  1.65748 val_accuracy=  0.49219
Epoch: 1514 val_loss=  1.70605 val_accuracy=  0.54297
Epoch: 1515 val_loss=  1.68139 val_accuracy=  0.50781
Epoch: 1516 val_loss=  1.62507 val_accuracy=  0.50391
Epoch: 1517 val_loss=  1.40890 val_accuracy=  0.55469
Epoch: 1518 val_loss=  1.51357 val_accuracy=  0.53906
Epoch: 1519 val_loss=  1.71338 val_accuracy=  0.48828
Epoch: 1520 val_loss=  1.38348 val_accuracy=  0.57812
Epoch: 1521 val_loss=  1.67959 val_accuracy=  0.56641
Epoch: 1522 val_loss=  1.64900 val_accuracy=  0.52734
Epoch: 1523 val_loss=  1.47982 val_accuracy=  0.60547
Epoch: 1524 val_loss=  1.75206 val_accuracy=  0.48438
Epoch: 1525 val_loss=  1.59740 val_accuracy=  0.54297
Epoch: 1526 val_loss=  1.58459 val_accuracy=  0.55469
Epoch: 1527 val_loss=  1.43927 val_accuracy=  0.57422
Epoch: 1528 val_loss=  1.72886 val_accuracy=  0.53906
Epoch: 1529 val_loss=  1.63233 val_accuracy=  0.53906
Epoch: 1530 val_loss=  1.640

Epoch: 1665 val_loss=  1.66193 val_accuracy=  0.53906
Epoch: 1666 val_loss=  1.56717 val_accuracy=  0.55469
Epoch: 1667 val_loss=  1.53867 val_accuracy=  0.58984
Epoch: 1668 val_loss=  1.40325 val_accuracy=  0.60938
Epoch: 1669 val_loss=  1.67634 val_accuracy=  0.51953
Epoch: 1670 val_loss=  1.52692 val_accuracy=  0.55469
Epoch: 1671 val_loss=  1.69392 val_accuracy=  0.51562
Epoch: 1672 val_loss=  1.68407 val_accuracy=  0.52344
Epoch: 1673 val_loss=  1.63639 val_accuracy=  0.52734
Epoch: 1674 val_loss=  1.73276 val_accuracy=  0.50781
Epoch: 1675 val_loss=  1.70360 val_accuracy=  0.53906
Epoch: 1676 val_loss=  1.66077 val_accuracy=  0.51172
Epoch: 1677 val_loss=  1.48536 val_accuracy=  0.55859
Epoch: 1678 val_loss=  1.44299 val_accuracy=  0.57031
Epoch: 1679 val_loss=  1.55170 val_accuracy=  0.57031
Epoch: 1680 val_loss=  1.57810 val_accuracy=  0.57812
Epoch: 1681 val_loss=  1.52090 val_accuracy=  0.55469
Epoch: 1682 val_loss=  1.62459 val_accuracy=  0.55469
Epoch: 1683 val_loss=  1.509

Epoch: 1817 val_loss=  1.72340 val_accuracy=  0.47656
Epoch: 1818 val_loss=  1.61443 val_accuracy=  0.53516
Epoch: 1819 val_loss=  1.58446 val_accuracy=  0.56250
Epoch: 1820 val_loss=  1.42358 val_accuracy=  0.58203
Epoch: 1821 val_loss=  1.76126 val_accuracy=  0.51953
Epoch: 1822 val_loss=  1.60158 val_accuracy=  0.55078
Epoch: 1823 val_loss=  1.66771 val_accuracy=  0.52734
Epoch: 1824 val_loss=  1.77925 val_accuracy=  0.51172
Epoch: 1825 val_loss=  1.52286 val_accuracy=  0.54297
Epoch: 1826 val_loss=  1.61433 val_accuracy=  0.51172
Epoch: 1827 val_loss=  1.40860 val_accuracy=  0.59375
Epoch: 1828 val_loss=  1.50211 val_accuracy=  0.57031
Epoch: 1829 val_loss=  1.42050 val_accuracy=  0.56641
Epoch: 1830 val_loss=  1.69744 val_accuracy=  0.50000
Epoch: 1831 val_loss=  1.70861 val_accuracy=  0.50000
Epoch: 1832 val_loss=  1.83550 val_accuracy=  0.48438
Epoch: 1833 val_loss=  1.60385 val_accuracy=  0.57031
Epoch: 1834 val_loss=  1.42770 val_accuracy=  0.59766
Epoch: 1835 val_loss=  1.677

In [10]:
# calculate the accuracy over the whole validation set

In [11]:
############## Validation and test ###############################
# with tf.device('/device:GPU:0'):
#     placeholders_val= {'dropout': tf.placeholder_with_default(0., shape=()),
#                        'is_training': tf.placeholder_with_default(True, shape=())}
#     feed_dict= construct_feed_dict(param_dict['dropout'], param_dict['is_training'], placeholders_val)

#     # Create model
#     model = GCN(holder, param_dict, input_dim=tf.shape(features_new)[2], placeholders_val= placeholders_val, logging=True)
#     model.output_avg()
#     save_freq= param_dict["save_freq"]
#     valid_freq= 391

    

# sess= tf.Session(config=tf.ConfigProto(allow_soft_placement= True))    
# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(coord=coord, sess=sess)
# print("data feat_batch, label_batch, meta_batch: ", file= open('printout','a'))

# # initialize the queue threads to start to shovel data
# # initialize the variable

# ALL_EVAL_ACC= []
# ALL_EVAL_LOSS= []


# sess.run(tf.initializers.global_variables())
# model.load(sess, save_path='./model-save/tmp/')

# dev_steps= int(np.ceil(500/256))
# for step in range(dev_steps):
#     # Train model
#     t = time.time()
    

#     feed_dict.update({placeholders_val['dropout']: param_dict_eval['dropout']})
#     feed_dict.update({placeholders_val['is_training']: param_dict_eval['is_training']})
    
#     outs_val = sess.run([model.loss_eval, model.accuracy_eval, 
#                          model.inputs_eval, model.outputs_eval, model.inter_out_eval], feed_dict= feed_dict)
#     ALL_EVAL_LOSS.append(outs_val[0])
#     ALL_EVAL_ACC.append(outs_val[1])
    
#     # Print results
#     print("Epoch:", '%04d' % (step + 1),"val_loss= ", "{:.5f}".format(outs_val[0]),
#           "val_accuracy= ", "{:.5f}".format(outs_val[1]))
    
# print("Optimization Finished!")
# print('total validation loss: ', sum(ALL_EVAL_LOSS)/len(ALL_EVAL_LOSS))
# print('total validation accuracy: ', sum(ALL_EVAL_ACC)/len(ALL_EVAL_ACC))


# # We request our child threads to stop ...
# coord.request_stop()
# # ... and we wait for them to do so before releasing the main thread
# coord.join(threads)
